# Preparing Dataset for Admissions Dashboard

In this notebook, I create a fake dataset for the Admissions Dashboard project. The dataset will represent admissions data of an imaginative business school. 

The imaginative business school is located in Europe, offering 8 academic programs in business studies leading to a Master of Science or MBA degree. Master's programs are designed for young people with little or no work experience, while MBA programs are designed for professionals with several years of work experience. All programs are taught in English, which attracts students from all over the world. Applications for upcoming intakes are accepted and processed throughout the year. Start dates may vary from program to program. Most programs start in the fall (in September or October), but there are some programs starting in the spring. 

For simplicity, I will assume that applications for each intake can be submitted during the 12 months preceding the month of the start of classes, i.e. applications for the "September 2022" are open from the beginning of September 2021 to the end of August 2022.  Applications that receive an "on hold" or "deferred" status may also be included in the application pool for the next intake. The submitted_on dates may fall between October 2020 and March 2024.

Applications for past intakes may have one of the following statuses representing outcomes of the admissions process: "withdrawn", "on_hold", "rejected", "declined_offer", "enrolled", "deferred". Applications for upcoming intakes can also have interim statuses "new", "review_in_progress" and "pending_offer".

In [4]:
import pandas as pd
import numpy as np
import os

In [3]:
# List of cohorts
cohorts = (
    [" ".join([program, "September", str(year)]) for year in [2021, 2022, 2023, 2024] for program in ['Master in Management',  'Master in Finance',  'Master in Marketing', 'Full-time MBA']] 
    + [" ".join([program, "October", str(year)]) for year in [2021, 2022, 2023, 2024] for program in ['Part-time MBA',  'Executive MBA']]
    + ["Master in Business Analytics October 2024"] + ["Online MBA September 2022"]
    + [" ".join(["Online MBA", month, str(year)]) for year in [2023, 2024] for month in ["March", "September"]]
) 
cohorts.sort()
cohorts

['Executive MBA October 2021',
 'Executive MBA October 2022',
 'Executive MBA October 2023',
 'Executive MBA October 2024',
 'Full-time MBA September 2021',
 'Full-time MBA September 2022',
 'Full-time MBA September 2023',
 'Full-time MBA September 2024',
 'Master in Business Analytics October 2024',
 'Master in Finance September 2021',
 'Master in Finance September 2022',
 'Master in Finance September 2023',
 'Master in Finance September 2024',
 'Master in Management September 2021',
 'Master in Management September 2022',
 'Master in Management September 2023',
 'Master in Management September 2024',
 'Master in Marketing September 2021',
 'Master in Marketing September 2022',
 'Master in Marketing September 2023',
 'Master in Marketing September 2024',
 'Online MBA March 2023',
 'Online MBA March 2024',
 'Online MBA September 2022',
 'Online MBA September 2023',
 'Online MBA September 2024',
 'Part-time MBA October 2021',
 'Part-time MBA October 2022',
 'Part-time MBA October 2023',

In [21]:
# Read data that were generated in Mockaroo
mock_data_folder = "../dataset/mock_data/"
df_mock_data = pd.concat([pd.read_csv(mock_data_folder + file_name, index_col= 0) for file_name in os.listdir(mock_data_folder)]).reset_index(drop=True)
df_mock_data

,gender,program,submitted_on,age_on_program_start,months_work_experience,nationality,other_nationality,applicant_state,years_work_experience
0,Male,Master in Finance,2020-12-16,20,0.0,India,NaN,deferred,NaN
1,Male,Master in Management,2021-02-10,23,18.0,India,NaN,declined_offer,NaN
2,Female,Master in Finance,2021-06-15,22,0.0,Taiwan,NaN,rejected,NaN
3,Female,Master in Management,2023-01-06,22,0.0,Germany,NaN,on_hold,NaN
4,Male,Master in Marketing,2023-10-26,23,0.0,Germany,NaN,rejected,NaN
...,...,...,...,...,...,...,...,...,...
3873,Male,Master in Business Analytics,2023-09-04,24,0.0,Norway,NaN,rejected,NaN
3874,Female,Master in Business Analytics,2023-09-16,21,0.0,Taiwan,NaN,declined_offer,NaN
3875,Female,Master in Business Analytics,2024-03-21,22,0.0,Germany,NaN,enrolled,NaN
3876,Male,Master in Business Analytics,2023-10-17,31,4.0,India,NaN,rejected,NaN


In [17]:
def assign_intake(df, start_month, fraction, *programs):
    """
    This function changes the "intake" column in the dataframe. 
    
    Input:
        df is a dataframe,
        start_month is the number of the month [1 - 12] when the program starts,
        *programs are the names of the programs where the function has bo be applied and the intake has to be changed.
        fraction is the fraction [0.01 - 1] of rows (after the *programs filter has been applied) where the intake has to be changed,
        
    The "intake" value is calculated based on the start_month passed to the function and the sumbitted_on column. 
    For example, the value "September 2022" is set if start_month = 9 and the sumbitted_on date is between September 2021 and August 2022. 
    
    Output: the dataframe where the "intake" column has been changed.
    
    """
    
    submitted_on_month = df.submitted_on.dt.month
    submitted_on_year = df.submitted_on.dt.year
    intake_year = submitted_on_year.where(submitted_on_month < start_month, submitted_on_year+1)
    intake_name = pd.to_datetime(intake_year.astype(str) + "-" +str(start_month) + "-01").dt.strftime('%B %Y')
    
    df_sample = df[df.program.isin(programs)].sample(frac = fraction, random_state = 1)
    
    return df.assign(intake = intake_name.where(df.index.isin(df_sample.index), df.intake))

def add_previous_year_data(df):
    """
    This function just creates a copy of all rows that record data for applications submitted more than a year ago,
    changes the record date and appends these rows to the original dataframe. 
    """
    current_date = df.submitted_on.max()
    same_date_previous_year = current_date -  pd.to_timedelta(366, unit='d')
    df_prev_year = df.loc[df.submitted_on <= same_date_previous_year].assign(date = same_date_previous_year)
    
    return pd.concat([df_prev_year, df]).reset_index()



df_dataset = (df_mock_data
              .astype({
                 "gender" : "category",
                 "submitted_on" : "datetime64",
                 "program" : "category",
                 "nationality" : "category",
                 "other_nationality" : "category"                 
              })
              .assign(intake = None) # create an empty column
              .pipe(assign_intake, 9, 1, 'Master in Marketing', 'Master in Finance', 'Master in Management', 'Full-time MBA', 'Online MBA' )
              .pipe(assign_intake, 10, 1, 'Master in Business Analytics',  'Part-time MBA', 'Executive MBA')
              .pipe(assign_intake, 3, 0.45, 'Online MBA')
              
              .assign(cohort = lambda df_: (df_.program.astype("str") + " " + df_.intake.astype("str")))
              .loc[lambda df_: df_.cohort.isin(cohorts)] 
              .assign(date = lambda df_: df_.submitted_on.max()) # date when data were recorded
              
              .pipe(add_previous_year_data)
              
              .assign(submitted_days_ago = lambda df_: (df_.date - df_.submitted_on).dt.days)
              .assign(applicant_state = lambda df_: df_.applicant_state
                      .where(df_.submitted_days_ago.gt(5), "new")
                      .where(~df_.submitted_days_ago.between(5,19), "review_in_progress")
                      .where((~df_.submitted_days_ago.between(20,39)) | df_.applicant_state.isin(["on_hold", "rejected"]), "pending_offer")
                     )
              .assign(applicant_id = lambda df_: df_["index"] + 1).drop(columns = ["index"])
              .assign(years_work_experience = lambda df_: df_.years_work_experience.where(df_.years_work_experience.notna(), df_.months_work_experience/12))
              .assign(months_work_experience = lambda df_: df_.months_work_experience.where(df_.months_work_experience.notna(), df_.years_work_experience*12))
              .astype({
                  "intake" : "category",
                  "cohort" : "category",
                  "applicant_state" : "category"
              }))

In [18]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6232 entries, 0 to 6231
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   gender                  6232 non-null   category      
 1   program                 6232 non-null   category      
 2   submitted_on            6232 non-null   datetime64[ns]
 3   age_on_program_start    6232 non-null   int64         
 4   months_work_experience  6232 non-null   float64       
 5   nationality             6232 non-null   category      
 6   other_nationality       458 non-null    category      
 7   applicant_state         6232 non-null   category      
 8   years_work_experience   6232 non-null   float64       
 9   intake                  6232 non-null   category      
 10  cohort                  6232 non-null   category      
 11  date                    6232 non-null   datetime64[ns]
 12  submitted_days_ago      6232 non-null   int64   

In [20]:
# Export dataset to csv
df_dataset[['date','applicant_id', 'program', 'intake', 'cohort','gender', 
       'age_on_program_start', 'months_work_experience', 'years_work_experience', 'nationality',
       'other_nationality', 'applicant_state', 
         'submitted_on', 'submitted_days_ago' ]].to_csv("../dataset/business_school_admissions_dataset.csv")